In [1]:
!python3 -c "import tensorflow as tf; tf.enable_eager_execution(); print(tf.reduce_sum(tf.random_normal([999, 9999])))"

2019-07-18 18:14:54.935410: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-07-18 18:14:55.646972: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x557918978300 executing computations on platform CUDA. Devices:
2019-07-18 18:14:55.647008: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-16GB, Compute Capability 7.0
2019-07-18 18:14:55.650106: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100160000 Hz
2019-07-18 18:14:55.658080: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55791903c630 executing computations on platform Host. Devices:
2019-07-18 18:14:55.658095: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-07-18 18:14:55.659050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 

In [2]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [3]:
tf.test.is_gpu_available()

True

In [4]:
if tf.test.gpu_device_name() == '/device:GPU:0':
  print("Using a GPU")
else:
  print("Using a CPU")

Using a GPU


In [5]:
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4024254152465233954
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 10780885942879961341
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12567651201342837307
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15651091252
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10922718326167081695
physical_device_desc: "device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:03:00.0, compute capability: 7.0"
]


In [6]:
with tf.device('/device:GPU:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [7]:
# Creates a graph.
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

[[22. 28.]
 [49. 64.]]


In [8]:
#!pwd

In [9]:
import pandas as pd
data = pd.read_csv("HeLa/ayu/new/P value 0.05.csv")
data.head(2)
data.shape

(19, 34)

In [10]:
df=pd.DataFrame(data)
X = df.iloc[:, :32]
y = df.iloc[:, 33]

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
list(le.classes_)
values=le.transform(y)
print(values)

[2 2 2 2 2 2 2 2 1 1 1 1 0 0 0 0 0 0 0]


In [12]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(le.transform(y).reshape(-1,1))
enc.categories_
#print(enc.transform(le.transform(y).reshape(-1,1)))

[array([0, 1, 2])]

In [13]:
#print(np.array(tf.one_hot(le.transform(y), depth=3)))
#vocabulary_feature_column =  tf.feature_column.categorical_column_with_vocabulary_list(key=["P", "V", "K"],vocabulary_list=["P", "V", "K"])
import numpy as np
n_values = np.max(values) + 1
Y=np.eye(n_values)[values]

In [14]:
#Y=pd.DataFrame(np.array(tf.one_hot(le.transform(y), depth=3)))
Y=pd.DataFrame(Y)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1, random_state = 0)
X_train.to_csv("X_train.csv", index = False, header = None)
X_test.to_csv("X_test.csv", index = False, header = None)
Y_train.to_csv("Y_train.csv", index = False, header = None)
Y_test.to_csv("Y_test.csv", index = False, header = None)
Y_test.head()

,0,1,2
10,0.0,1.0,0.0
1,0.0,0.0,1.0


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train.shape[1],Y_train.shape[1] 

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.


(32, 3)

In [16]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [17]:
classifier = Sequential()
classifier.add(Dense(units = 9, activation = 'relu', input_dim=X_train.shape[1]))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = Y_train.shape[1] , activation = 'sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [18]:
classifier.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [19]:
classifier.fit(X_train, Y_train, batch_size = X_train.shape[1],epochs = 100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
17/17 [==============================] - 1s 68ms/step - loss: 0.3649 - acc: 0.1176
Epoch 2/100
17/17 [==============================] - 0s 195us/step - loss: 0.3613 - acc: 0.1176
Epoch 3/100
17/17 [==============================] - 0s 179us/step - loss: 0.3578 - acc: 0.1765
Epoch 4/100
17/17 [==============================] - 0s 167us/step - loss: 0.3544 - acc: 0.1765
Epoch 5/100
17/17 [==============================] - 0s 160us/step - loss: 0.3510 - acc: 0.1765
Epoch 6/100
17/17 [==============================] - 0s 156us/step - loss: 0.3479 - acc: 0.2353
Epoch 7/100
17/17 [==============================] - 0s 158us/step - loss: 0.3450 - acc: 0.2353
Epoch 8/100
17/17 [==============================] - 0s 163us/step - loss: 0.3422 - acc: 0.2353
Epoch 9/100
17/17 [==============================] - 0s 160us/step - loss: 0.3393 - acc: 0.2353
Epoch 10/100
17/17 [==============================] - 0s 167us/step - loss: 0.3365 - acc:

In [20]:
Y_pred = classifier.predict(X_test)
#Y_pred
y_pred=(Y_pred.argmax(1)[:,None] == np.arange(Y_pred.shape[1])).astype(int)
y_pred

array([[0, 1, 0],
       [0, 0, 1]])

In [21]:
y_test=Y_test.values
y_test=(y_test.argmax(1)[:,None] == np.arange(y_test.shape[1])).astype(int)
y_test

array([[0, 1, 0],
       [0, 0, 1]])

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
cm

array([[1, 0],
       [0, 1]])